In [113]:
from geopy.geocoders import BaiduV3
from geopy import distance
import pygeohash as gh
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyecharts.charts import *
from pyecharts import options as opts
import datetime
import pygeohash as gh
import geopandas as gpd
from shapely.geometry import Point,LineString,MultiPoint
%matplotlib inline

In [108]:
fgpd = gpd.read_file(r"E:\Spatiotemporal_Data\摩拜共享单车2017年8月数据集\mobike_shanghai_sample_updated.csv") # 读取数据
data = gpd.GeoDataFrame(fgpd)

In [86]:
data.drop_duplicates()  # 去除重复的行

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,geometry
0,78387,158357,10080,2016/8/20 6:57,121.348,31.389,2016/8/20 7:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",None
1,891333,92776,6605,2016/8/29 19:09,121.508,31.279,2016/8/29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",None
2,1106623,152045,8876,2016/8/13 16:17,121.383,31.254,2016/8/13 16:36,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",None
3,1389484,196259,10648,2016/8/23 21:34,121.484,31.32,2016/8/23 21:43,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",None
4,188537,78208,11735,2016/8/16 7:32,121.407,31.292,2016/8/16 7:41,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",None
...,...,...,...,...,...,...,...,...,...,...,...
102356,1479550,214335,1423,2016/8/28 18:03,121.478,31.297,2016/8/28 18:09,121.481,31.304,"121.478,31.297#121.479,31.297#121.480,31.297#1...",None
102357,1478273,160487,3067,2016/8/14 20:22,121.32,31.238,2016/8/14 20:28,121.312,31.235,"121.312,31.235#121.313,31.235#121.313,31.236#1...",None
102358,367733,179530,12746,2016/8/27 9:54,121.391,31.307,2016/8/27 10:06,121.398,31.306,"121.390,31.307#121.391,31.307#121.392,31.307#1...",None
102359,64915,167419,837,2016/8/20 6:04,121.515,31.269,2016/8/20 6:10,121.51,31.272,"121.510,31.272#121.510,31.273#121.511,31.273#1...",None


In [87]:
# 轨迹数据矢量化
points = []
lines = []
for i in range(len(data)):
    track = data.loc[i,'track'].split('#')
    try:
        for j in range(len(track)):
            lon,lat = track[j].split(',')
            if lat[-1] == '\\':
                lat = lat[:-1]
            point = [float(lon),float(lat)]
            points.append(point)
    except:
        print(i,data.loc[i,'track'].split('#'))
    line = MultiPoint(points)
    lines.append(line)
    points.clear()
lines

 ...]

In [45]:
lat = '31.26\\'  #  \\为一个字符
lat[-1]

'\\'

In [109]:
data['geometry'] = lines
# data.plot()

In [110]:
# data = data[data['start_time'].str.split(' ')[0]=='2016/8/1']
a = data['start_time'].str.split(' ',expand=True)  # expand=True将结果保存为dataframe
data = data[gpd.pd.to_datetime(a[0])<'2016-8-2']

In [111]:
data.sort_values(by='start_time', ascending=True, na_position='first')

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,geometry
91307,31355,79699,759,2016/8/1 0:23,121.52,31.309,2016/8/1 0:32,121.525,31.316,"121.520,31.309#121.521,31.309#121.522,31.309#1...","MULTIPOINT (121.52000 31.30900, 121.52100 31.3..."
15161,31388,33279,1440,2016/8/1 0:25,121.505,31.294,2016/8/1 0:40,121.517,31.309,"121.504,31.295#121.504,31.296#121.505,31.294#1...","MULTIPOINT (121.50400 31.29500, 121.50400 31.2..."
67939,31401,3060,5463,2016/8/1 0:26,121.375,31.107,2016/8/1 0:41,121.387,31.103,"121.375,31.107#121.376,31.107#121.376,31.108#1...","MULTIPOINT (121.37500 31.10700, 121.37600 31.1..."
94977,31423,43119,4666,2016/8/1 0:32,121.528,31.263,2016/8/1 0:52,121.531,31.282,"121.527,31.265#121.527,31.266#121.528,31.263#1...","MULTIPOINT (121.52700 31.26500, 121.52700 31.2..."
47303,31438,116238,2193,2016/8/1 0:33,121.434,31.249,2016/8/1 0:46,121.427,31.26,"121.427,31.260#121.428,31.259#121.428,31.260#1...","MULTIPOINT (121.42700 31.26000, 121.42800 31.2..."
...,...,...,...,...,...,...,...,...,...,...,...
7017,293465,21284,3667,2016/8/1 9:56,121.453,31.187,2016/8/1 10:04,121.442,31.191,"121.442,31.189#121.442,31.190#121.442,31.191#1...","MULTIPOINT (121.44200 31.18900, 121.44200 31.1..."
4739,293484,78535,991,2016/8/1 9:57,121.405,31.13,2016/8/1 10:08,121.393,31.132,"121.393,31.132#121.393,31.133#121.394,31.133#1...","MULTIPOINT (121.39300 31.13200, 121.39300 31.1..."
25600,293495,120052,5209,2016/8/1 9:58,121.403,31.17,2016/8/1 10:01,121.404,31.165,"121.403,31.167#121.403,31.168#121.403,31.169#1...","MULTIPOINT (121.40300 31.16700, 121.40300 31.1..."
13275,293521,125662,6398,2016/8/1 9:59,121.369,31.217,2016/8/1 10:16,121.353,31.22,"121.352,31.220#121.353,31.220#121.354,31.220#1...","MULTIPOINT (121.35200 31.22000, 121.35300 31.2..."


In [112]:
data.to_file(r'E:\Spatiotemporal_Data\摩拜共享单车2017年8月数据集\multipoint_8_1.shp',encoding='utf-8')

C:\Users\LMQ\AppData\Local\Temp\ipykernel_6792\1104527175.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  data.to_file(r'E:\Spatiotemporal_Data\摩拜共享单车2017年8月数据集\multipoint_8_1.shp',encoding='utf-8')
